In [ ]:
def solve_1D_BPP_WB(cargo, deferred_items = None, attempted_combinations = None , number_of_opened_uld = None, packed_ULDs = None, open_new_uld = False, open_new_extra_uld = False, deferred_items_optimal_flag = False):
    """
    Solve 1D Bin Packing Problem
    """
    m = Model("1D_BPP")

    loadfactor = 0.8

    '''Open a new extra ULD compared to the actual set of ULDs [Feedback loop]'''

    if open_new_extra_uld is True:
        max_index = max([j.index for j in cargo.uld])
        extra_PMC = ULD(max_index + 1, 'PMC', 'PMC' + '-' + str(max_index + 1))
        extra_AKE = ULD(max_index + 2, 'AKE', 'AKE' + '-' + str(max_index + 2))
        cargo.uld.append(extra_PMC)
        cargo.uld.append(extra_AKE)
        cargo.define_parameters_ULD()

    '''Decision variables'''

    u = {}
    p = {}

    for j in cargo.uld:
        u[j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'u_{j.index}')

    for i in cargo.items:
        for j in cargo.uld:
            p[i.index, j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'p_{i.index}_{j.index}')

    '''Feedback loop actions'''

    for uld, items in packed_ULDs.items():
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT and str(uld) == str(j.serialnumber):
                    u[j.index].lb = 1
                    for i in cargo.items:
                        p[i.index, j.index].lb = 0
                        p[i.index, j.index].ub = 0
                    for item in items:
                        for i in cargo.items:
                            if str(i.serialnumber) == str(item):
                                p[i.index, j.index].lb = 1
                                p[i.index, j.index].ub = 1

    
    if deferred_items is not None and open_new_uld == False:
        for j, items in deferred_items.items():
            for i in items:
                p[i.index, j.index].lb = 0
                p[i.index, j.index].ub = 0

    m.update()

    '''Model Sense'''

    m.ModelSense = GRB.MINIMIZE

    m.update()

    # BUG 11: Indices de multi-objetivo no secuenciales, este modelo usa: 1, 2, 3, 4 (en 1D-BPP) y luego 0, 5 (en W&B).
    '''Objective function 2 --> Place larger volume item in PMC/PAG ULDs and small volume item in AKE ULDs [Medium Priority]'''
    volume_threshold = data_analysis.threshold_volume_in_AKE() * 1000000

    score_AKE = {}
    score_PMC_PAG = {}

    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            score_AKE[j.index] = m.addVar(vtype=GRB.CONTINUOUS, name=f'score_AKE_{j.index}', lb=0)
            score_PMC_PAG[j.index] = m.addVar(vtype=GRB.CONTINUOUS, name=f'score_PMC_PAG_{j.index}', lb=0)

            m.addConstr(score_AKE[j.index] == quicksum(p[i.index, j.index] * (volume_threshold - i.volume) for i in cargo.items if i.volume < volume_threshold and 'AKE' in j.type))
            m.addConstr(score_PMC_PAG[j.index] == quicksum(p[i.index, j.index] * (volume_threshold - i.volume) for i in cargo.items if i.volume < volume_threshold and ('PMC' in j.type or 'PAG' in j.type)))
    
    obj_volume_preference = quicksum(score_AKE[j.index] + score_PMC_PAG[j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT)
    m.setObjectiveN(obj_volume_preference, index = 1, priority = 4, weight = 1)

    m.update()

    ''' Objective function 2 --> Minimize number of ULDs opened [Medium Priority] '''

    obj2 = quicksum(u[j.index] for j in cargo.uld)
    m.setObjectiveN(obj2, index = 2, priority = 3, weight = 1)

    m.update()

    '''Objective function 3 --> Minimize underutilization of ULDs  [Medium Priority]'''
   
    min_load_factor_threshold = 0.2

    underutilization_penalty = {}
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            actual_load_factor = quicksum(i.volume * p[i.index, j.index] for i in cargo.items) / j.volume

            loadfactor_underutilization = m.addVar(vtype=GRB.CONTINUOUS, name=f"shortfall_volume_{j.index}", lb=0)
            m.addConstr(loadfactor_underutilization >= min_load_factor_threshold - actual_load_factor)

            underutilization_penalty[j.index] = loadfactor_underutilization
    
    obj_underutilization = quicksum(underutilization_penalty.values())
    m.setObjectiveN(obj_underutilization, index=3, priority=2, weight=1)

    m.update()

    '''Objective function 4 --> Minimize the separation of items with the same serialnumber prefix over different ULDs [Medium Priority]'''

    booking_groups = cargo.get_prefix_groups()

    Y = {} # Separation penalty, para contar ULDs, no binary penalty como antes erroneamente
    Z = {}
    for b_i, items in booking_groups.items():
        Y[b_i] = m.addVar(vtype=GRB.INTEGER, lb=0, name=f'Y_{b_i}')
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT:
                Z[b_i, j.index] = m.addVar(vtype=GRB.BINARY, name=f'Z_{b_i}_{j.index}')

    obj_separation = quicksum(Y[b_i] for b_i in booking_groups.keys())
    m.setObjectiveN(obj_separation, index=4, priority=1, weight=1)

    m.update()

    for b_i, items in booking_groups.items():
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT:
                for i in items:
                    m.addConstr(
                        p[i.index, j.index] <= Z[b_i, j.index],
                        name=f'C_P_Z_{b_i}_{i.index}_{j.index}'
                    )

    for b_i in booking_groups.keys():
        m.addConstr(
            Y[b_i] == quicksum(Z[b_i, j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT),
            name=f'C_Y_Z_{b_i}'
        )


    '''Constraints'''

    # R1: Apertura de ULDs - Minimum number of ULDs to open (feedback loop)
    m.addConstr(quicksum(u[j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT) >= number_of_opened_uld, name = 'open_new_uld_constraint')

    # R2: Uso de ULD - If ULD is opened, at least one item must be placed in it
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            m.addConstr(quicksum(p[i.index, j.index] for i in cargo.items) >= u[j.index], name=f'C_new_uld_{j.index}')

    # R3: Capacidad de Peso - Weight of items in ULD cannot exceed ULD max weight capacity
    for j in cargo.uld:
            m.addConstr(quicksum(i.weight * p[i.index, j.index] for i in cargo.items) <= j.max_weight * u[j.index], name = f'C1_{j.index}')

    # R4: Capacidad Volumétrica - Volume of items in ULD cannot exceed ULD volume with loadfactor
    for j in cargo.uld:
            m.addConstr(quicksum(i.volume * p[i.index, j.index] for i in cargo.items) <= j.volume * u[j.index] * loadfactor, name = f'C2_{j.index}')

    # BUG 2: Asignacion incluye BAX/BUP/T 
    # Ver Model.ipynb linea 211: quicksum(p[i.index, j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT) == 1

    # R5: Asignación Única - Every item must be placed in exactly one ULD
    for i in cargo.items:
        m.addConstr(quicksum(p[i.index, j.index] for j in cargo.uld) == 1, name = f'C3_{i.index}')  # BUG 2: Falta filtro isNeitherBAXnorBUPnorT

    # R6: Prohibición en BAX/BUP/T - No items can be loaded in BAX, BUP, or T ULDs
    for i in cargo.items:
        for j in cargo.uld:
            if j.isBAXorBUPorT:
                m.addConstr(p[i.index, j.index] == 0, name = f'C_combi_3_{i.index}_{j.index}')

    # R7: Manejo Especial COL/CRT - COL and CRT items cannot be in same ULD
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            for i_1 in cargo.items:
                for i_2 in cargo.items:
                    if i_1 != i_2:
                        if i_1.COL == 1 and i_2.CRT == 1:
                            m.addConstr(p[i_1.index, j.index] + p[i_2.index, j.index] <= 1, name = f'C_special_1_{i_1.index}_{i_2.index}_{j.index}')


    if not deferred_items_optimal_flag:
        volume_env = m.getMultiobjEnv(1)
        uld_env = m.getMultiobjEnv(2)
        underutilization_env = m.getMultiobjEnv(3)
        separation_env = m.getMultiobjEnv(4)

        volume_env.setParam(GRB.Param.TimeLimit, 15)
        uld_env.setParam(GRB.Param.TimeLimit, 15)
        separation_env.setParam(GRB.Param.TimeLimit, 15)
        underutilization_env.setParam(GRB.Param.TimeLimit, 15)

        m.optimize()
    
    else:
        m.setParam(GRB.Param.MIPGap, 0.1)
        volume_env = m.getMultiobjEnv(1)
        uld_env = m.getMultiobjEnv(2)
        underutilization_env = m.getMultiobjEnv(3)
        separation_env = m.getMultiobjEnv(4)

        volume_env.setParam(GRB.Param.TimeLimit, 15)
        uld_env.setParam(GRB.Param.TimeLimit, 15)
        separation_env.setParam(GRB.Param.TimeLimit, 15)
        underutilization_env.setParam(GRB.Param.TimeLimit, 15)

        m.optimize()

    status = m.status
    if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE: 
        print('The model is infeasible or unbounded')
        # m.computeIIS()
        # m.write('model.ilp')
        print('Infeasibility report written to model.ilp')
    elif status == GRB.Status.TIME_LIMIT:
        print('Time limit reached')
    elif status == GRB.Status.OPTIMAL:
        print('Solution found')
    elif status == GRB.Status.INTERRUPTED:
        print('Optimization was stopped early')

    '''Results'''

    results_1D_BPP_WB = {}

    for j in cargo.uld:
        if u[j.index].x == 1:
            if j.isNeitherBAXnorBUPnorT:
                results_1D_BPP_WB[j] = []

    for i in cargo.items:
        for j in cargo.uld:
            if p[i.index, j.index].x > 0.999:
                results_1D_BPP_WB[j].append(i)

    number_of_items_in_results = 0
    for j, items in results_1D_BPP_WB.items():
        number_of_items_in_results += len(items)

    loadfactor_dict = {}
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            volume_loadfactor = (sum(i.volume * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.volume)
            loadfactor_dict[j] = volume_loadfactor

    return results_1D_BPP_WB, loadfactor_dict


In [2]:
def solve_3D_BPP(results_1D_BPP_WB):
    deferred_items = {}
    all_placed_items = {}
    all_extreme_points = {}

    for j, items in results_1D_BPP_WB.items():
        extreme_points = EP.get_starting_extreme_points(j)
        items_to_place = items.copy()
        placed_items = {}
        deferred_items[j] = []

        while items_to_place:
            next_item, placement_details, defer_reason = EP.find_best_next_item_and_placement(items_to_place, j, placed_items, extreme_points)

            if next_item:
                best_ep, best_orientation, best_merit, best_support_count = placement_details
                placed_items, added_points, removed_points = EP.place_item(next_item, best_ep, placed_items, best_orientation, extreme_points)
                extreme_points = EP.update_extreme_points(extreme_points, placed_items[next_item.serialnumber], next_item, j)
                items_to_place.remove(next_item)
                          
            else:
                print(f'Items deferred for ULD {j.serialnumber}:')
                for item in items_to_place:
                    specific_reason = defer_reason or 'Unknown reason'
                    print(f'{item.serialnumber} deferred due to {defer_reason}')
                    deferred_items[j].append(item)
                print('---------------------------------------------------------------------------')

                items_to_place = []

        all_placed_items[j] = placed_items
        all_extreme_points[j] = extreme_points

    return deferred_items, all_placed_items, all_extreme_points

In [3]:
def feedback_loop(cargo):
    total_time_1D_BPP = 0
    total_time_3D_BPP = 0
    iteration = 1
    number_of_opened_uld = 0
    open_new_uld = False
    open_new_extra_uld = False
    extra_uld_opened_in_previous_iteration = False
    solution_found = False
    deferred_items = {}
    previous_deferred_items = {}
    packed_ULDs = {}
    attempted_combinations = set()

    color_map = EP.get_color_map(cargo.items)
    folder_path = project_setup.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline = True, optimized_actual = False)
    number_of_uld_actually_used = len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])



    while not solution_found:
        print(f'Iteration {iteration}')
        print('===========================================================================')
        print(f'Starting optimization for iteration {iteration}')
        start_time_1D_BPP = time.time()
        results_1D_BPP, loadfactor_dict = solve_1D_BPP_WB(cargo, deferred_items = deferred_items, attempted_combinations = attempted_combinations, 
                                                                                                                   number_of_opened_uld = number_of_opened_uld, packed_ULDs = packed_ULDs, 
                                                                                                                   open_new_uld = open_new_uld, open_new_extra_uld = open_new_extra_uld,
                                                                                                                   deferred_items_optimal_flag = False)
        open_new_extra_uld = False
        total_time_1D_BPP += time.time() - start_time_1D_BPP
        number_of_opened_uld = len(results_1D_BPP)

        start_time_3D_BPP = time.time()
        deferred_items, placed_items, extreme_points = solve_3D_BPP(results_1D_BPP)
        total_time_3D_BPP += time.time() - start_time_3D_BPP

        for j, items in deferred_items.items():
            if ((len(items) == 0) and (loadfactor_dict[j] >= 0.65)) or ((loadfactor_dict[j] >= 0.75) and (len(deferred_items[j]) <= 2)):
                if len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT]) - len(packed_ULDs) > 2:
                    packed_ULDs[j.serialnumber] = list(placed_items[j].keys())


        number_of_deferred_items = sum(len(items) for items in deferred_items.values())

        if number_of_deferred_items == 0:
            solution_found = True
            print('Solution found with no deferred items')

        if solution_found == False:
            print('---------------------------------------------------------------------------')
            print(f'Number of deferred items: {number_of_deferred_items}')
            print(f'Number of opened ULDs: {number_of_opened_uld}') 
            print('---------------------------------------------------------------------------')

        # for j, items in deferred_items.items():
        #     combination = (j, tuple(results_1D_BPP[j]))
        #     attempted_combinations.add(combination)

        deferred_items = deferred_items

        
        for j, items in deferred_items.items():
            for i in items:
                previous_deferred_items[i.serialnumber] = previous_deferred_items.get(i.serialnumber, 0) + 1

                if ((previous_deferred_items[i.serialnumber] >= (number_of_opened_uld - len(packed_ULDs.keys()))) or 
                    ((number_of_opened_uld < len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])) and (sum(1 for value in deferred_items.values() if value) == 1)) and
                    not extra_uld_opened_in_previous_iteration):
                    print(f'Flag set: opening a new ULD')
                    print(f'Item {i.serialnumber} is causing the flag')
                    print('---------------------------------------------------------------------------')
                    if number_of_opened_uld < number_of_uld_actually_used:
                        number_of_opened_uld = number_of_opened_uld + 1
                        previous_deferred_items = {k: 0 for k in previous_deferred_items.keys()} 
                        extra_uld_opened_in_previous_iteration = False
                    elif number_of_opened_uld >= number_of_uld_actually_used:
                        open_new_extra_uld = True
                        number_of_opened_uld = number_of_opened_uld + 1
                        print(number_of_opened_uld)
                        previous_deferred_items = {k: 0 for k in previous_deferred_items.keys()} 
                        extra_uld_opened_in_previous_iteration = True
                    break

            iteration += 1
        
        if solution_found:
            plot.BPP(cargo, results_1D_BPP, placed_items, extreme_points, color_map, folder_path)

    return results_1D_BPP, total_time_1D_BPP, total_time_3D_BPP

In [ ]:
def solve_WB(cargo, results_3D_BPP):
    """
    Solve Weight and Balance
    """
    m = Model("WB")

    start_time_WB = time.time()
    a_lat_TOW = 0.5
    b_lat_TOW = 0.5
    a_lat_LW = 0.5
    b_lat_LW = 0.5

    new_cargo_uld = [j for j in cargo.uld if j.isBAXorBUPorT]

    for j, items in results_3D_BPP.items():
        new_cargo_uld.append(j)
        j.weight = sum([i.weight for i in items])

    cargo.uld = new_cargo_uld

    '''Decision variables'''

    f = {}
    
    for j in cargo.uld:
        for t in aircraft.loadlocations:
            f[j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'f_{j.index}_{t.index}')


    m.update()

    '''Model Sense'''

    m.ModelSense = GRB.MINIMIZE

    m.update()

    ''' Objective function 1 --> Maximize the %MAC [Highest Priority] '''

    ZFW_index_obj = m.addVar(vtype=GRB.CONTINUOUS, name="ZFW_index_obj")

    m.addConstr(ZFW_index_obj == aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4))
    
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= ZFW_index_obj, "ZFW_index_fwd_constraint")
    m.addConstr(ZFW_index_obj <= aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), "ZFW_index_aft_constraint")

    MAC_obj = (((aircraft.C * (ZFW_index_obj - aircraft.K)) / aircraft.ZFW) + aircraft.reference_arm - aircraft.lemac) / (aircraft.mac_formula / 100)


    m.setObjectiveN(MAC_obj, index = 0, priority = 2, weight = -1)

    m.update()

    '''Objective function 5 --> Minimize the proximity score of BAX ULDs [Low Priority]'''
    
    obj4 = quicksum(aircraft.define_proximity_score_loadlocation(t) * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations if j.isBAX)
    m.setObjectiveN(obj4, index = 5, priority = 1, weight = 1)

    '''Constraints'''
    
    # W1: Asignación de ULD - Every ULD must be assigned to exactly one position
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.loadlocations) == 1, name = f'C_combi_2_{j.index}')

    # W2: Posición Única - Each position can hold at most one ULD
    for t in aircraft.loadlocations:
        m.addConstr(quicksum(f[j.index, t.index] for j in cargo.uld) <= 1, name = f'C4_{t.index}')

    # W3: Posiciones Prohibidas - ULD type must match position type compatibility
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.define_forbidden_positions_for_ULD(j)) == 0, name = f'C5_{j.index}')

    #Because one type of ULDs’ predefined positions may overlap with others, one of these overlapping positions is occupied, and the others can no longer be allocated. 
    for j_1 in cargo.uld:
        for j_2 in cargo.uld:
            if j_1 != j_2:
                for t_1 in aircraft.loadlocations:
                        for t_2 in aircraft.define_overlapping_positions(t_1):
                            m.addConstr(f[j_1.index, t_1.index] + f[j_2.index, t_2.index] <= 1, name = f'C6_{j_1.index}_{j_2.index}_{t_1.index}_{t_2.index}')

    # W5: Peso por Posición - Weight at position cannot exceed position limit
    for t in aircraft.loadlocations:
            m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.define_max_weight_postion(t), name = f'C7_2_{t.index}')

    # W6a: Peso Compartimento C1 - Total weight in compartment 1 limit
    m.addConstr(
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1)
        <= aircraft.max_weight_C1,
        name='C_Added_1'
    )

    # W6b: Peso Compartimento C2 - Total weight in compartment 2 limit
    m.addConstr(
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2)
        <= aircraft.max_weight_C2,
        name='C_Added_2'
    )

    # W6c: Peso Compartimento C3 - Total weight in compartment 3 limit
    m.addConstr(
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3)
        <= aircraft.max_weight_C3,
        name='C_Added_3'
    )

    # W6d: Peso Compartimento C4 - Total weight in compartment 4 limit
    m.addConstr(
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4)
        <= aircraft.max_weight_C4,
        name='C_Added_4'
    )
        
    # W6e: Peso Compartimentos C1+C2 - Combined weight in front compartments
    m.addConstr(
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1_C2)
        <= aircraft.max_weight_C1_C2,
        name='C_Added_5'
    )
        
    # W6f: Peso Compartimentos C3+C4 - Combined weight in rear compartments
    m.addConstr(
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3_C4)
        <= aircraft.max_weight_C3_C4,
        name='C_Added_6'
    )
        
    # W7: Peso Total (MPL) - Total cargo weight cannot exceed Maximum Payload Limit
    m.addConstr(quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld)
                <= aircraft.define_MPL(), name = f'C8')

    # W8: Balance Lateral TOW - Lateral balance for takeoff weight (left-right)
    m.addConstr(quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) -
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) <= 
                a_lat_TOW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name = f'C9_1')
    
    m.addConstr(quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) -
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld  for t_left in aircraft.loadlocations_left) <= 
                a_lat_TOW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name = f'C9_2')

    # W9: Balance Lateral LW - Lateral balance for landing weight (left-right)
    m.addConstr(quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) - 
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) <= 
                a_lat_LW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name = f'C10_1')

    m.addConstr(quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) - 
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) <= 
                a_lat_LW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name = f'C10_2')

    # W10: Envelope CG TOW - Longitudinal CG envelope for takeoff (forward and aft limits)
    m.addConstr(aircraft.define_INDEX_TOW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) + 
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4), name = 'C11_1')
    
    m.addConstr(aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4) <= 
                aircraft.define_INDEX_TOW_aft(aircraft.aircraft_type), name = 'C11_2')
    
    # W11: Envelope CG ZFW - Longitudinal CG envelope for zero fuel weight (forward and aft limits)
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4), name = 'C12_1')
    
    m.addConstr(aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4) <= 
                aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), name = 'C12_2')
    # W12: COL/CRT Especial - COL and CRT cargo cannot be in the same compartment (aircraft-specific)
    uld_with_COL = []
    uld_with_CRT = []

    for j in cargo.uld:
        col_flag = getattr(j, 'COL', 0) == 1
        crt_flag = getattr(j, 'CRT', 0) == 1
        if results_3D_BPP and j in results_3D_BPP:
            col_flag = col_flag or any(getattr(item, 'COL', 0) == 1 for item in results_3D_BPP[j])
            crt_flag = crt_flag or any(getattr(item, 'CRT', 0) == 1 for item in results_3D_BPP[j])
        if col_flag:
            uld_with_COL.append(j)
        if crt_flag:
            uld_with_CRT.append(j)

    if str(aircraft.aircraft_type) in ['772', '77W']:
        compartments = {
            'front': aircraft.loadlocations_C1_C2,
            'aft': aircraft.loadlocations_C3_C4,
        }
        for label, load_locations in compartments.items():
            col_var = m.addVar(vtype=GRB.BINARY, name=f'COL_{label}')
            crt_var = m.addVar(vtype=GRB.BINARY, name=f'CRT_{label}')

            if uld_with_COL:
                m.addConstr(
                    quicksum(f[j.index, t.index] for j in uld_with_COL for t in load_locations)
                    <= len(uld_with_COL) * col_var,
                    name=f'C_special_COL_{label}'
                )
            else:
                m.addConstr(col_var == 0, name=f'C_special_COL_{label}_zero')

            if uld_with_CRT:
                m.addConstr(
                    quicksum(f[j.index, t.index] for j in uld_with_CRT for t in load_locations)
                    <= len(uld_with_CRT) * crt_var,
                    name=f'C_special_CRT_{label}'
                )
            else:
                m.addConstr(crt_var == 0, name=f'C_special_CRT_{label}_zero')

            m.addConstr(col_var + crt_var <= 1, name=f'C_special_COL_CRT_conflict_{label}')

    if str(aircraft.aircraft_type) in ['789', '781']:
        if uld_with_COL:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_COL for t in aircraft.loadlocations_C3_C4) == 0,
                name='C_special_COL_aft_787'
            )
        if uld_with_CRT:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_CRT for t in aircraft.loadlocations_C3_C4) == 0,
                name='C_special_CRT_aft_787'
            )

        col_front = m.addVar(vtype=GRB.BINARY, name='COL_front_787')
        crt_front = m.addVar(vtype=GRB.BINARY, name='CRT_front_787')

        if uld_with_COL:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_COL for t in aircraft.loadlocations_C1_C2)
                <= len(uld_with_COL) * col_front,
                name='C_special_COL_front_787'
            )
        else:
            m.addConstr(col_front == 0, name='C_special_COL_front_787_zero')

        if uld_with_CRT:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_CRT for t in aircraft.loadlocations_C1_C2)
                <= len(uld_with_CRT) * crt_front,
                name='C_special_CRT_front_787'
            )
        else:
            m.addConstr(crt_front == 0, name='C_special_CRT_front_787_zero')

        m.addConstr(col_front + crt_front <= 1, name='C_special_COL_CRT_conflict_front_787')

    
    '''Optimize the model'''
    WB_env = m.getMultiobjEnv(0)
    bax_env = m.getMultiobjEnv(5)

    WB_env.setParam(GRB.Param.TimeLimit, 60)
    bax_env.setParam(GRB.Param.TimeLimit, 15)

    m.optimize()


    status = m.status
    if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE: 
        print('The model is infeasible or unbounded')
        # m.computeIIS()
        # m.write('model.ilp')
        print('Infeasibility report written to model.ilp')
    elif status == GRB.Status.TIME_LIMIT:
        print('Time limit reached')
    elif status == GRB.Status.OPTIMAL:
        print('Solution found')
    elif status == GRB.Status.INTERRUPTED:
        print('Optimization was stopped early')


    '''Results'''
    results_filename = 'Results.txt'
    folder_path = project_setup.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline = True, optimized_actual = False)
    results_file_path = os.path.join(folder_path, results_filename)

    with open(results_file_path, 'w') as file:
        total_weight = 0
        number_of_uld_solution = 0
        for t in aircraft.loadlocations:
            for j in cargo.uld:
                if f[j.index, t.index].x > 0.9999:
                    number_of_uld_solution += 1
                    print(f'ULD {j.serialnumber} with weight {j.weight:.1f} kg is loaded to position {t.location}')
                    file.write(f'ULD {j.serialnumber} with weight {j.weight:.1f} kg is loaded to position {t.location}\n')


        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'Weight in Compartment 1: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1):.1f} kg')
        file.write(f'Weight in Compartment 1: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1):.1f} kg\n')
        print(f'Weight in Compartment 2: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2):.1f} kg')
        file.write(f'Weight in Compartment 2: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2):.1f} kg\n')
        print(f'Weight in Compartment 3: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3):.1f} kg')
        file.write(f'Weight in Compartment 3: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3):.1f} kg\n')
        print(f'Weight in Compartment 4: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4):.1f} kg')
        file.write(f'Weight in Compartment 4: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4):.1f} kg\n')

        #Calculating %MAC
        TOW_index = aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX()
        ZFW_index = aircraft.DOI + aircraft.define_INDEX_PAX()

        dict_loadlocations = {aircraft.delta_index_cargo_C1: aircraft.loadlocations_C1, aircraft.delta_index_cargo_C2: aircraft.loadlocations_C2, aircraft.delta_index_cargo_C3: aircraft.loadlocations_C3, aircraft.delta_index_cargo_C4: aircraft.loadlocations_C4}

        for j in cargo.uld:
            for value, compartment in dict_loadlocations.items():
                for t in compartment:
                    if f[j.index, t.index].x > 0.9999:
                        TOW_index += j.weight * f[j.index, t.index].x * float(value) 
                        ZFW_index += j.weight * f[j.index, t.index].x * float(value)

        increment_value = aircraft.define_ff_increment_MAC_ZFW(MAC_obj.getValue())
        fuel_saving_kg = aircraft.TripF * (increment_value / 100)

        number_of_uld_solution = number_of_uld_solution - len([j for j in cargo.uld if j.isBAXorBUPorT])

        print('===========================================================================')
        file.write('===========================================================================\n')
        print(f'%MAC ZFW is {MAC_obj.getValue()}')
        file.write(f'%MAC ZFW is {MAC_obj.getValue()}\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}')
        file.write(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}\n')
        print(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg')
        file.write(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'{number_of_uld_solution} ULDs are built by the model')
        file.write(f'{number_of_uld_solution} ULDs are built by the model\n')
        print(f'{cargo.total_number_of_build_ULDs} ULDs were actually built')
        file.write(f'{cargo.total_number_of_build_ULDs} ULDs were actually built\n')
        print('---------------------------------------------------------------------------')

        plot.WB(aircraft, ZFW_index, TOW_index, folder_path)

        total_time_WB = time.time() - start_time_WB

    return total_time_WB, folder_path

In [5]:
def solve_baseline(cargo):
    results_3D_BPP, total_time_1D_BBP, total_time_3D_BPP = feedback_loop(cargo)
    total_time_WB, folder_path = solve_WB(cargo, results_3D_BPP)

    model_info_path = os.path.join(folder_path, 'Model_Information.txt')

    with open(model_info_path, 'w') as file:
        file.write(f'Total time: {total_time_1D_BBP + total_time_3D_BPP + total_time_WB:.3f} seconds\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time 1D BPP: {total_time_1D_BBP:.3f} seconds\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time 3D BPP: {total_time_3D_BPP:.3f} seconds\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time WB: {total_time_WB:.3f} seconds\n')
    

In [ ]:
solve_baseline(cargo)